# PySTAC Client exempel

Dokumentation: https://pystac-client.readthedocs.io/en/stable/index.html

In [ ]:
from pystac_client import Client
from copy import deepcopy

import geopandas as gpd
import pandas as pd
from shapely.geometry import shape

In [ ]:
# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i["geometry"] = shape(_i["geometry"])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ["properties.datetime", "properties.created", "properties.updated"]:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index("properties.datetime", inplace=True)
    return gdf

Sätt inloggningsuppgifter och skapa klient:

In [ ]:
import base64
import pystac_client

# encode credentials
user_name = "Be not afraid"
password = "hemligt"
userpass = f"{user_name}:{password}"
b64_userpass = base64.b64encode(userpass.encode()).decode()

# API url
base_url = "https://api.lantmateriet.se/stac-hojd/v1"

# create the client
client = pystac_client.Client.open(
    url=base_url,
    headers={
        'Authorization': f"Basic {b64_userpass}"
    }
)

## Sök collections
Dokumentation: https://pystac-client.readthedocs.io/en/stable/api.html#pystac_client.Client.search

In [ ]:
search = client.search(max_items=10)
items = list(search.items_as_dicts())
len(items)

In [ ]:
items_gdf = items_to_geodataframe(items)

## Visualisera

In [ ]:
import folium
# Create minimal dataset
minimal_gdf = items_gdf[['geometry']].copy()
minimal_gdf = minimal_gdf.set_crs(epsg=4326, allow_override=True)

# Project to a suitable projected CRS (Web Mercator) for centroid calculation
projected_gdf = minimal_gdf.to_crs(epsg=3857)
center_point = projected_gdf.geometry.centroid.to_crs(epsg=4326)

# Create folium map using the properly calculated center
m = folium.Map(location=[center_point.y.mean(), 
                        center_point.x.mean()],
               zoom_start=10)

# Add GeoDataFrame to the map (make sure it's in WGS84/EPSG:4326)
folium.GeoJson(minimal_gdf.to_crs(epsg=4326)).add_to(m)
m

## Hämta data med Rasterio

In [ ]:
import rasterio
from rasterio.plot import show

In [ ]:
# change this to 'data' when we get correct permissions
ASSET_TYPE = 'thumbnail'

In [ ]:
import rasterio.plot
with rasterio.open(items[0]['assets'][ASSET_TYPE]['href']) as dataset:
    rasterio.plot.show(dataset)